In [ ]:
import numpy as np
import matplotlib.pyplot as plt

N = 5

x1 = np.random.random(N)
x2 = x1 + [np.random.randint(10)/10 for i in range(N)]
C1 = [x1,x2]

x1 = np.random.random(N)
x2 = x1 - [np.random.randint(10)/10 for i in range(N)] - 0.1
C2 = [x1,x2]

f = [0,1]

w = np.array([-0.3,0.3])
for i in range(N):
    x = np.array([C2[0][i], C2[1][i]])
    y = np.dot(w,x)
    if y >= 0:
         print("Class C1")
    else:
         print("Class C2")

plt.scatter(C1[0][:], C1[1][:], s=10, c='red')
plt.scatter(C2[0][:], C2[1][:], s=10, c='blue')
plt.plot(f)
plt.grid(True)
plt.show()

In [11]:
import numpy as np

def f(x):
    return 2/(1 + np.exp(-x)) - 1
def df(x):
    return 0.5*(1+x)*(1-x)

W1 = np.array([[-0.3, 0.1,0.4],
               [-0.1,0.5,-0.4]])
W2 = np.array([0.1,-0.4])

def go_forward(inp):
    sum = np.dot(W1, inp)
    out = np.array([f(x) for x in sum])

    sum = np.dot(W2, out)
    y = f(sum)
    return (y, out)

def train(epoch):
    global W2, W1
    lmd = 0.01
    N = 10000
    count = len(epoch)
    for k in range(N):
        x = epoch[np.random.randint(0, count)]
        y, out = go_forward(x[0:3])
        e = y - x[-1]
        delta = e*df(y)
        W2[0] = W2[0] - lmd * delta * out[0]
        W2[1] = W2[1] - lmd * delta * out[1]
        
        delta2 = W2*delta*df(out)

        W1[0, :] = W1[0, :] - np.array(x[0:3]) * delta2[0] * lmd
        W1[1, :] = W1[1, :] - np.array(x[0:3]) * delta2[1] * lmd

epoch = [(-1, -1, -1, -1),
         (-1, -1, 1, 1),
         (-1, 1, -1, -1),
         (-1, 1, 1, 1),
         (1, -1, -1, -1),
         (1, -1, 1, 1),
         (1, 1, -1, -1),
         (1, 1, 1, 1)]

train(epoch)

for x in epoch:
    y, out = go_forward(x[0:3])
    print(f"Выходное значение НС: {y} => {x[-1]}")

Выходное значение НС: -0.9346391460947872 => -1
Выходное значение НС: 0.9379753713709149 => 1
Выходное значение НС: -0.9345965166278849 => -1
Выходное значение НС: 0.9360938349252215 => 1
Выходное значение НС: -0.9360938349252215 => -1
Выходное значение НС: 0.9345965166278849 => 1
Выходное значение НС: -0.9379753713709148 => -1
Выходное значение НС: 0.9346391460947874 => 1


In [1]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.leyers import Dense


ModuleNotFoundError: No module named 'tensorflow.keras.leyers'